In [ ]:
#There are many ways to handle security, authentication and authorization
#And it normally is a complex and "difficult" topic

#FastAPI provides several tools to help with Security easily, rapidly in a standard way without having to study and learn all the security specifications.


#OAuth2 - is a specification that defines several ways to handle authentication and authorization, It is what all the systems with login with facebook google twitter github use underneath




Security - First Steps

In [2]:
#Here we see how to add authorization to api requests


import nest_asyncio
import uvicorn

nest_asyncio.apply()


from typing import Annotated
from fastapi import Depends, FastAPI
from fastapi.security import OAuth2PasswordBearer

app = FastAPI()


oauth2_scheme = OAuth2PasswordBearer(tokenUrl = "token")


@app.get("/items/")
async def read_items(token: Annotated[str, Depends(oauth2_scheme)]):
    return {"token": token}


uvicorn.run(app, port=8000)

INFO:     Started server process [15296]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:53090 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:53090 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:53100 - "POST /token HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:41424 - "GET /items/ HTTP/1.1" 401 Unauthorized


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [15296]


The password flow

In [ ]:
#The password flow is one of the ways/flows defined in OAuth2, to handle security and authentication.

#OAuth2 was designed so that the backedn or API could be independent of the server that authenticates the user


#But in this case, the same  FastAPI application will handle the API and the authentication.

#So lets review it from that simplified point of view:


    # The user types the username and password in the frontend and hits Enter
    
    # The frontend sends that username and password to a specific URL in our API

    # The API  checks that username and password, and responds with a token(we haven't implemented any of this yet)


            #A token is just a string with some content that we can use later to verify this user
            #Normally a token is set to expire after some time
                #So the user will have to log in again at some point later.
                #And if teh token is stolen the risk is less. It is not like a permanent key that will work forever(in most of the cases).

    #The frontend stores that token temporarily somewhere

    #The user clicks in the frontend to go to another section of the frontend web app.

    #The frontend needs to fetch some more data from the API.
        #But it needs authentication for that specific endpoint.
        #So to authenticate with our API, it sends a header Authorization with a value of Bearer plus the token
        #If the token contains foobar, the content of the Authorization header would be:
            #Bearer foobar


            

FastAPI's OAuth2PasswordBearer

In [4]:
# we can use teh oauth2_scheme in a dependency with depends


from typing import Annotated

from fastapi import Depends, FastAPI
from fastapi.security import OAuth2PasswordBearer


app = FastAPI()

oauth2_scheme = OAuth2PasswordBearer(tokenUrl = "token")


@app.get('/items/')
async def read_items(token: Annotated[str, Depends(oauth2_scheme)]):
    return {'token': token}


uvicorn.run(app, port=8000)


INFO:     Started server process [15296]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:37076 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:37076 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:43612 - "GET /items/ HTTP/1.1" 401 Unauthorized


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [15296]


## Get Current User

In [ ]:
from typing import Annotated

from fastapi import Depends, FastAPI
from fastapi.security import OAuth2PasswordBearer


app = FastAPI()


oauth2_scheme = OAuth2PasswordBearer(tokenUrl = "token")

@app.get('/items/')
async def read_items(token: Annotated[str, Depends(oauth2_scheme)]):
    return {"token": token}


#But that is still not that useful



In [5]:
#Hence Create a user model

from typing import Annotated

from fastapi import Depends, FastAPI
from fastapi.security import OAuth2PasswordBearer

from pydantic import BaseModel


app = FastAPI()


oauth2_scheme = OAuth2PasswordBearer(tokenUrl = 'token')


class User(BaseModel):
    username: str
    email: str | None = None
    full_name: str | None = None
    disabled: bool | None = None


def fake_decode_token(token):
    return User(username = token + "fakedecoded", email = 'john@example.com', full_name = 'John Lamare')


async def get_current_user(token: Annotated[str, Depends(oauth2_scheme)]):
    user = fake_decode_token(token)
    return user

@app.get('/users/me')
async def read_users_me(current_user: Annotated[User, Depends(get_current_user)]):
    return current_user


uvicorn.run(app, port=8000)

INFO:     Started server process [15296]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:59312 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:59312 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [15296]


## Simple OAuth2 with Password and Bearer

In [9]:
#scope
#--- The spec also says that the client can send another form field "scope" separated by spaces


#Each scope is just a string.

#Code to get the username and password
#--->


from typing import Annotated

from fastapi import Depends, FastAPI,  HTTPException, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from pydantic import BaseModel


fake_users_db = {
    'johndoe':{
        'username': 'johndoe',
        'fullname': 'John doe',
        'email': 'johndoe@example.com',
        'hashed_password': 'fakehashedsecret',
        'disabled': False,
    },

    'alice':{
        'username': 'alice',
        'full_name': 'Alice Wonderson',
        'email': 'alice@example.com',
        'hashed_password': 'fakehashedsecret2',
        'disabled': True,
    },
}


app = FastAPI()


def fake_hash_password(password: str):
    return 'fakehashed' + password



oauth2_scheme = OAuth2PasswordBearer(tokenUrl = 'token')


class User(BaseModel):
    username: str
    email: str | None = None
    full_name: str | None = None
    disabled: bool | None = None


class UserInDB(User):
    hashed_password: str


def get_user(db, username: str):
    if username in db:
        user_dict = db[username]
        return UserInDB(**user_dict)


def fake_decode_token(token):
    #This doesnt provide any security at all
    #Check the next version
    user = get_user(fake_users_db, token)
    return user


async def get_current_user(token: Annotated[str, Depends(oauth2_scheme)]):
    user = fake_decode_token(token)
    if not user:
        raise HTTPException(
            status_code = status.HTTP_401_UNAUTHORIZED,
            detail = 'Invalid authentication credentials',
            headers = {'WWW-Authenticate': 'Bearer'},
        )
    return user


async def get_current_active_user(
    current_user: Annotated[User, Depends(get_current_user)]
):
    if current_user.disabled:
        raise HTTPException(status_code = 400, detail='Inactive user')
    return current_user

@app.post('/token')
async def login(form_data: Annotated[OAuth2PasswordRequestForm, Depends()]):
    user_dict = fake_users_db.get(form_data.username)
    if not user_dict:
        raise HTTPException(status_code = 400, detail='Incorrect username or password')
    user = UserInDB(**user_dict)
    hashed_password = fake_hash_password(form_data.password)
    if not hashed_password == user.hashed_password:
        raise HTTPException(status_code = 400, detail="Incorrect username or password")

    return {"access_token": user.username, "token_type": "bearer"}

@app.get('/users/me')
async def read_users_me(
    current_user: Annotated[User, Depends(get_current_active_user)]
):
    return current_user


uvicorn.run(app, port=8000)

INFO:     Started server process [15296]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56112 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:51022 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:51022 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:46888 - "POST /token HTTP/1.1" 200 OK
INFO:     127.0.0.1:39480 - "POST /token HTTP/1.1" 200 OK
INFO:     127.0.0.1:59508 - "GET /users/me HTTP/1.1" 200 OK


In [8]:
#Return the token

#The response of teh token endpoint must be a json object.

#It should have a token_type In our case as we are using bearer tokens, the token type should be bearer

#And it should have an access_token with a string containing our access token

#For this simple example we are going to just be completely insecure and return the same username as the token


#------------------
# Byt eh spce you should return a JSON with an access_token and a token_type the same as in this example. This is something that you have to do yourself in your code, nd make sure you use those JSON keys.

#its almost the only thing that you have to remember to do correctly yourself, to be compliant with the specifications.

#For the rest, FastAPI handles it for you.

## Update the dependencies

In [ ]:
#Now we are going to update our dependencies

#We want to get the current_user only if this user is active

#So, we create an additonal dependency get_current_active_user that in turn uses get_current_user as a dependency

#Both of these dependencies will just return an HTTP error if the user doesnt exist, or if is inactive.

#So, in our endpoint, we will only get a user if the user exists, was correctly authenticated, and is active:

